In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))
townships

,Unnamed: 0,project_id,latitude,longitude,township,state,area,median_psf,median_price,poi_nearby_education,poi_nearby_transportation,poi_nearby_healthcare,nearest_school,nearest_train_station,nearest_hospital,total_poi_nearby,nearest_poi,transaction_count,coordinates
0,1,28054,1.850838,103.098501,Taman Melewar 2,Johor,Parit Raja,32.5,140000.0,7,0,3,672.717277,118274.719516,16503.783674,10,672.717277,2,"(1.8508384999999998, 103.0985005)"
1,3,19000,2.424192,103.837082,Taman Mersing,Johor,Mersing,92.5,290000.0,10,0,2,566.989543,179114.418087,1143.523032,12,566.989543,4,"(2.4241923, 103.8370822)"
2,4,19009,2.502480,102.831990,"Taman Mewah, Sungai Segamat",Johor,Segamat,58.0,160000.0,17,0,4,458.392988,67423.562590,1919.519018,21,458.392988,3,"(2.50248, 102.83199)"
3,5,19036,1.989595,102.873294,"Taman Mutiara, Parit Sulong",Johor,Parit Sulong,147.0,368800.0,7,0,2,645.808917,88877.182185,15580.830078,9,645.808917,3,"(1.9895953000000002, 102.8732944)"
4,9,28046,2.109806,103.396274,Taman Perdana,Johor,Kluang,102.0,235000.0,4,0,1,442.303113,135761.690074,10285.212878,5,442.303113,4,"(2.1098055000000002, 103.39627409999999)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6757,7466,23082,3.016821,101.623228,Bandar Puteri Puchong,Selangor,Bandar Puteri Puchong,535.5,965000.0,17,4,3,866.352847,1111.662286,781.983448,24,781.983448,154,"(3.0168207000000002, 101.6232283)"
6758,7467,11372,3.053950,101.528999,"Seksyen 19, Shah Alam",Selangor,Shah Alam,271.0,200000.0,22,2,3,351.342108,496.611056,1374.577130,27,351.342108,155,"(3.0539500713347993, 101.52899932861)"
6759,7468,11479,3.130250,101.473999,Seri Jati & Baiduri,Selangor,Setia Alam/Alam Nusantara,NaN,NaN,8,0,0,1597.334823,7114.680424,5884.831166,8,1597.334823,0,"(3.13025, 101.47399899999999)"
6760,7469,10419,2.876342,101.806980,Bandar Bukit Mahkota Bangi,Selangor,Bangi,251.0,385000.0,6,0,1,710.395361,3877.793774,8023.283116,7,710.395361,159,"(2.8763422999999997, 101.80697959999999)"


In [46]:
from geopy.distance import distance

# calculate distance between two points
def calc_distance(source, target):
    return(distance(source, target).m)


# # find nearby city
# def nearby_city(target_coordinate):
#     townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
#     townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))
    
#     townships['distance'] = townships['coordinates'].apply(
#         lambda x: calc_distance(x, target_coordinate)
#     )

#     # return all cities within 15km
#     nearby_cities = townships[townships['distance'] <= 5000][
#         ['area', 'distance']
#     ].groupby('area').min().reset_index().sort_values('distance')

#     return (json.dumps(list(nearby_cities['area'])))


def nearby_townships(target_coordinate):
    # target_coordinate = (request.args.get('lat'), request.args.get('lng'))
    townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
    transactions = pd.read_csv('../data/processed/edgeprop_transactions_processed.csv')
    
    townships = pd.merge(
        townships, 
        transactions.drop_duplicates(subset=['project_id']), 
        on='project_id', 
        how='inner', 
        suffixes=('', '_2')
    )

    townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))

    townships['distance'] = townships['coordinates'].apply(
        lambda x: calc_distance(x, target_coordinate)
    )

    # return all townships within 2km
    nearby_townships_2km = townships[townships['distance'] <= 2000][
        ['project_id', 'township', 'distance']
    ].groupby('project_id').min().reset_index().sort_values('distance')

    return (json.dumps(nearby_townships_2km.head(15).to_dict('records')))



In [47]:
nearby_cities = nearby_townships((3.0596886793921034, 101.686018355625))

In [48]:
nearby_cities

'[{"project_id": 9273, "township": "Covillea", "distance": 122.97702323695884}, {"project_id": 13737, "township": "Vista Komanwel A", "distance": 149.68150217165712}, {"project_id": 15266, "township": "Vista Komanwel C", "distance": 158.6446380546851}, {"project_id": 11318, "township": "Savanna 1", "distance": 403.8534326254633}, {"project_id": 14994, "township": "Sri Endah Apartment", "distance": 735.2439422927349}, {"project_id": 25143, "township": "Bukit Jalil", "distance": 751.0910102775507}, {"project_id": 8743, "township": "Arena Green", "distance": 751.9290302010766}, {"project_id": 12299, "township": "Taman Esplanad", "distance": 786.8425105496823}, {"project_id": 9621, "township": "Endah Villa Condominium", "distance": 838.7291944464837}, {"project_id": 8596, "township": "8 Petaling", "distance": 875.5156462222857}, {"project_id": 28205, "township": "The Link 2", "distance": 1083.3221900315584}, {"project_id": 16569, "township": "Residensi jalil Idaman @ Twin Arkz", "distance"

In [25]:
transactions = pd.read_csv('../data/processed/edgeprop_transactions_processed.csv')
transactions.info()
townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
townships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155841 entries, 0 to 155840
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 155841 non-null  int64  
 1   project_id                 155841 non-null  int64  
 2   township                   155841 non-null  object 
 3   property_type              155841 non-null  object 
 4   tenure                     155841 non-null  object 
 5   area_sqft                  155841 non-null  float64
 6   non_landed                 155841 non-null  int64  
 7   bedrooms                   155841 non-null  float64
 8   psf                        155841 non-null  float64
 9   price                      155841 non-null  int64  
 10  planning_region            155841 non-null  object 
 11  state                      155841 non-null  object 
 12  city                       155841 non-null  object 
 13  median_psf                 15

In [6]:
from geopy.distance import distance

# calculate distance between two points
def calc_distance(source, target):
    return(distance(source, target).m)


# find nearby city
def nearest_poi(target_coordinate):
    # target_coordinate = (request.args.get('lat'), request.args.get('lng'))

    pois = pd.read_csv('../data/processed/iproperty_pois_preprocessed.csv')
    pois['coordinates'] = list(zip(pois['latitude'], pois['longitude']))
    
    pois['distance'] = pois['coordinates'].apply(
        lambda x: calc_distance(x, target_coordinate)
    )

    return(pois['distance'].min())





In [7]:
nearest_poi((3.064520, 101.665436))

156.40462573036172

In [8]:
transactions = pd.read_csv('../data/processed/edgeprop_transactions_processed.csv')

In [9]:
transactions['property_type'].unique()

array(['Flat', 'Condominium/Apartment', 'Town House', 'Terrace House',
       'Hotel/Service Apartment', 'Detached House', 'Semi-Detached House',
       'Low-Cost House', 'Cluster House'], dtype=object)

In [10]:
transactions['tenure'].unique()

array(['Freehold', 'Leasehold'], dtype=object)

In [11]:
import pickle
model = pickle.load(open('../data/model.pkl', 'rb'))


In [18]:
input_features = pd.DataFrame([
    json.loads('{"nearest_poi": "151.04681088991547", "project_id": "15265", "property_type": "Condominium/Apartment", "tenure": "Freehold", "area_sqft": "1230", "bedrooms": "3"}')
])
# predicted_price = model.predict(input_features)

In [13]:
def state_lookup(township_id):
  township_id = int(township_id)
  townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv').drop_duplicates(
    subset='project_id', keep="first"
  )

  results = townships[townships['project_id'] == township_id]

  return results['state'].iloc[0]


def city_lookup(township_id):
  township_id = int(township_id)
  townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv').drop_duplicates(
    subset='project_id', keep="first"
  )

  results = townships[townships['project_id'] == township_id]

  return results['area'].iloc[0]


In [14]:
# input_features['state'] = input_features['project_id'].apply(state_lookup)
# input_features['city'] = input_features['project_id'].apply(city_lookup)
input_features['project_id'] = input_features['project_id'].astype('int')

In [19]:
input_features

,nearest_poi,project_id,property_type,tenure,area_sqft,bedrooms
0,151.04681088991547,15265,Condominium/Apartment,Freehold,1230,3


In [20]:
predicted_price = model.predict(input_features)

/home/kitlim/py3_kernel/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


ValueError: Found unknown categories ['15265'] in column 0 during transform

In [17]:
'{:,.2f}'.format(predicted_price[0])

'1,006,974.37'

In [21]:
transactions[transactions['project_id']=='15265']

/home/kitlim/py3_kernel/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Unnamed: 0,project_id,township,property_type,tenure,area_sqft,non_landed,bedrooms,psf,price,...,poi_nearby_education,poi_nearby_transportation,poi_nearby_healthcare,nearest_school,nearest_train_station,nearest_hospital,total_poi_nearby,nearest_poi,transaction_count,year
